# 데이터 전처리

In [40]:
import pandas as pd

START_MONTH = 7
END_MONTH = 12
df_rent = [None] * 13 # [n] = n월 대여이력
df_broken = [None] * 13 # [n] = n월 고장신고내역

<br>

## 데이터프레임 로드

In [41]:
df_rent[7] = pd.read_csv("서울특별시 공공자전거 대여이력 정보_2207.csv")
df_rent[8] = pd.read_csv("서울특별시 공공자전거 대여이력 정보_22081.csv")
df_rent[9] = pd.read_csv("서울특별시 공공자전거 대여이력 정보_2209.csv")
df_rent[10] = pd.read_csv("서울특별시 공공자전거 대여이력 정보_2210.csv")
df_rent[11] = pd.read_csv("서울특별시 공공자전거 대여이력 정보_2211.csv")
df_rent[12] = pd.read_csv("서울특별시 공공자전거 대여이력 정보_2212.csv")
df_broken[0] = pd.read_csv("서울시 공공자전거 고장신고 내역_22.07-12.csv")

<br>

## 대여이력 전처리

In [42]:
def pre_process_rent(df, month):
    df = df[(df['대여일시'] != '0000-00-00 00:00:00') & (df['반납일시'] != '0000-00-00 00:00:00')]
    df['대여일시'] = pd.to_datetime(df['대여일시'])
    df['반납일시'] = pd.to_datetime(df['반납일시'])
    df.drop(['대여거치대', '반납거치대', '대여대여소ID', '반납대여소ID'], axis=1, inplace=True)
    df = df[(df['성별'] == 'M') | (df['성별'] == 'F')]
    df = df[df['생년'] != '1901']
    df = df[df['생년'] != '\\N']
    df["반납대여소번호"] = df["반납대여소번호"].str.replace('\\','')
    df["반납대여소번호"] = df["반납대여소번호"].str.replace('N','-1')
    df['대여요일'] = df['대여일시'].dt.day_name()
    
    return df

for month in range(START_MONTH, END_MONTH + 1):
    df_rent[month] = pre_process_rent(df_rent[month], month)

C:\Users\imihl\AppData\Local\Temp\ipykernel_1256\964684185.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["반납대여소번호"] = df["반납대여소번호"].str.replace('\\','')
C:\Users\imihl\AppData\Local\Temp\ipykernel_1256\964684185.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["반납대여소번호"] = df["반납대여소번호"].str.replace('\\','')
C:\Users\imihl\AppData\Local\Temp\ipykernel_1256\964684185.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["반납대여소번호"] = df["반납대여소번호"].str.replace('\\','')
C:\Users\imihl\AppData\Local\Temp\ipyker

<br>

## 고장신고내역 전처리

In [43]:
df_broken[0]['등록일시'] = pd.to_datetime(df_broken[0]['등록일시'])
for month in range(START_MONTH, END_MONTH + 1):
    df_broken[month] = df_broken[0][pd.DatetimeIndex(df_broken[0]['등록일시']).month == month]
    df_broken[month] = df_broken[month].drop_duplicates('자전거번호',keep='last')#마지막 신고이력 남기기

<br>

## 고장데이터만 추출

고장신고 전 제일 최근의 대여이력을 추출하였습니다.

In [44]:
bad = [None] * 13
for month in range(START_MONTH, END_MONTH + 1):    
    broken = pd.merge(df_rent[month], df_broken[month],on='자전거번호')
    broken = broken[broken['대여일시'] <= broken['등록일시']] # 고장신고 이전에 대여된 이력
    broken = broken[broken['대여 대여소명'] != broken['반납대여소명']]# 같은곳에서 반납된것 드랍
    broken = broken[broken['이용시간(분)'] > 2] # 2분이하로 탄 것 드랍

    broken = broken[(broken['등록일시'] - broken['반납일시']) <= pd.to_timedelta(2, unit = 'D')]
    broken = broken.drop_duplicates(['자전거번호'],keep='last')
    broken['고장'] = 1
    
    bad[month] = broken

<br>

## 정상데이터만 추출

고장 수리후 첫번째, 두번째 대여이력을 추출하였습니다.

In [47]:
normal = [None] * 13
for month in range(START_MONTH, END_MONTH + 1):
    good = pd.merge(df_rent[month], df_broken[month],on='자전거번호')
    good = good[good['대여일시'] > good['등록일시']]
    good_first = good.drop_duplicates('자전거번호',keep='first')

    good_exceptfirst = pd.merge(good, good_first, how='outer', indicator=True).query("_merge == 'left_only'").drop(columns=['_merge'])
    good_second = good_exceptfirst.drop_duplicates('자전거번호',keep='first')
    
    good_firstAndSecond = pd.concat([good_first,good_second])
    good_firstAndSecond = good_firstAndSecond[good_firstAndSecond['대여 대여소명'] != good_firstAndSecond['반납대여소명']]#같은곳에서 반납된것 드랍
    good_firstAndSecond = good_firstAndSecond[good_firstAndSecond['이용시간(분)'] > 2]
    
    good_firstAndSecond['고장'] = 0
    
    normal[month] = good_firstAndSecond

<br>

## 고장, 정상데이터 병합

고장 13207개, 정상 13207개 총 26414개의 데이터입니다.

In [48]:
train_data = None
for month in range(START_MONTH, END_MONTH + 1):
    bad[month] = bad[month].sample(n=len(normal[month]))
    train = pd.concat([bad[month], normal[month]])
    if month == START_MONTH:
        train_data = train
    else:
        train_data = pd.concat([train_data, train])
    
train_data.sort_values('대여일시')
train_data.head()

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,반납일시,반납대여소번호,반납대여소명,이용시간(분),이용거리(M),생년,성별,이용자종류,대여요일,등록일시,고장구분,고장
178996,SPB-56738,2022-07-05 22:47:00,1452,겸재교 진입부,2022-07-05 23:20:00,604,답십리초등학교 옆 공원,33,4290.00,2005,M,내국인,Tuesday,2022-07-06 08:59:00,타이어,1
215151,SPB-46107,2022-07-02 20:18:00,4468,가락1동주민센터,2022-07-02 20:22:00,2648,헬리오시티 112동 앞,4,577.04,1978,F,내국인,Saturday,2022-07-02 20:23:00,안장,1
120595,SPB-61188,2022-07-08 01:22:00,1230,송파중학교 정문,2022-07-08 01:28:00,1282,송파소방서 맞은편(성내4교),6,1230.00,2002,F,내국인,Friday,2022-07-08 08:53:00,안장,1
144759,SPB-30387,2022-07-01 22:03:00,155,가좌역1 번출구 앞,2022-07-01 22:07:00,3131,연가교 교차로,4,765.88,1985,M,내국인,Friday,2022-07-02 03:20:00,기타,1
175744,SPB-59909,2022-07-08 07:44:00,3509,세종사이버대학교,2022-07-08 07:52:00,501,광진구의회 앞,7,1960.00,2004,M,내국인,Friday,2022-07-08 08:20:00,기타,1


<br>

## 파일변환

In [ ]:
train_data.to_csv("training_data7~12.csv", encoding='cp949', index=False)